In [ ]:
# BEGINNING OF CODE

# I begin by setting up my Planet API key as an environment variable
import os
import planet
os.environ['PL_API_KEY'] = 'PLAKce2e61fa7c9e4a8e9ae548de4caec1a2' 

# Here I put my coordinates and defined my GeoJSON geometry. My location is where I'm currently living, Winchester, Virginia.
planetgeojson = {
    "type": "Polygon", # my geometry type is a polygon since I have multiple coordinates
    "coordinates": [    # I input a coordinates I acquired from geojson.io of the AoI, that being my hometown of Winchester
    [ 
      [-78.20473661414968, 39.22822228510228],
      [-78.20473661414968, 39.12914301819299],
      [-78.07636138898725, 39.12914301819299],
      [-78.07636138898725, 39.22822228510228],
      [-78.20473661414968, 39.22822228510228]
    ]
  ]
  }
planetgeojson

# Here I set my filters so I can get specific imagery from Planet
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": planetgeojson
}

# code to get images acquired within a date range, in this case the 25th to 26th of September 2022.
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2022-09-25T00:00:00.000Z",
    "lte": "2022-09-26T00:00:00.000Z"
  }
}

# code to only get images which have <10% cloud coverage for clarity
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.1
  }
}

# here I combined the geo, date, cloud filters into one dict to send to the API
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}
combined_filter

# I import packages that I need to make requests and set my API key to a variable
import json
import requests
from requests.auth import HTTPBasicAuth
PLANET_API_KEY = os.getenv('PL_API_KEY')

# Next I set the item type to show imagery from PlanetScope sensors then I make a search request and view the results after
item_type = "PSScene"
item_type
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}
search_request
search_result = requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)
search_result
results = search_result.json()

# Here I extract the first image asset ID and put it into a file path so I can download it later
image_ids = [feature['id'] for feature in results['features']]
id0 = image_ids[0]
id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)
result = requests.get(
    id0_url,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
  )
result

# I check to make sure that the basic asset I've chosen is active or inactive
print(result.json()['ortho_analytic_4b']['status'])

# I put this information into user defined variables so I can get the activation link
links = result.json()[u"ortho_analytic_4b"]["_links"]
self_link = links["_self"]
activation_link = links["activate"]
activate_result = requests.get(
    activation_link,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
  )
activation_status_result = requests.get(
    self_link,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
  )
    
# Here I check to make sure the status shows active so it tells me I can download the image
print(activation_status_result.json()["status"])

download_link = activation_status_result.json()["location"]
print(download_link)